In [1]:
# 
root = "/home/shi/WorkSpace/projects/scMultiNet_Data/"
feature_locs = [
    "/cls0.pkl",
    "/cls1.pkl",
    "/cls2.pkl",
    "/cls3.pkl",
    "/cls4.pkl",
]

raw_data_loc = root + "/Step_1_data/Dataset_bulk_other/TrVal_dataset_Ground_truth.pkl"
code_loc = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"

save_folder = root + "/Step_4_data/"

ckpt_folder = root+"/Step_2_data/Cook/regression/ckpt/model_d0.3_0.629.ckpt"
ckpt_folder_cls = root + "/Step_2_data/Cook/classification/ckpt/model_0.9019531292560845.ckpt"
train_root = root + "/Step_1_data/Dataset_bulk_other/train/"

cls_nb = 2

In [2]:
import sys
sys.path.append(code_loc)

import dill
with open(raw_data_loc, "rb") as f:
    [trainset,valset,_,label_dict] = dill.load(f)


In [3]:
print(label_dict)

{'fake_lable_0': 0, 'fake_lable_1': 1}


In [4]:
# load model and calc features
import torch
import torch.nn.functional as F
import sys
DIM = 128


import torch.nn as nn
class FuseNet(nn.Module):
    def __init__(self,in_dim=128*5, dropout = 0., h_dim = 128, out_dim = 1, ):
        nn.Module.__init__(self,)

        self.fc1 = nn.Linear(in_features=in_dim, out_features=512, bias=True)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(in_features=512, out_features=h_dim, bias=True)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(in_features=h_dim, out_features=out_dim, bias=True)

    def forward(self, x, out_key:str = "all"):

        x = self.fc1(x)
        x = self.act1(x)
        x = self.dropout1(x)
        if out_key == "fc1":
            return x

        x = self.fc2(x)
        x = self.act2(x)
        x = self.dropout2(x)
        if out_key == "fc2":
            return x
        x = self.fc3(x)
        if out_key == "all":
            return x
        else:
            raise ValueError("out_key must be one of 'fc1', 'fc2', 'all'")

class FuseNet(nn.Module):
    def __init__(self,in_dim=128*5, dropout = 0., h_dim = 128, out_dim = 1, ):
        nn.Module.__init__(self,)

        self.fc1 = nn.Linear(in_features=in_dim, out_features=512, bias=True)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(in_features=512, out_features=1, bias=True)
        self.act2 = nn.ReLU()

    def forward(self, x, out_key:str = "all"):

        x = self.fc1(x)
        x = self.act1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.act2(x)
        return x

# 从文件夹中读取所有类的feature matrix
import pickle
import numpy as np


read_list = ["cls0","cls1","cls2","cls3","cls4"]
#-->train
train_feat_list = []
for i in range(len(read_list)):
    with open(train_root+read_list[i]+".pkl","rb") as f:
        [feat,label] = pickle.load(f)
        train_feat_list.append(feat)
train_label = label
print(len(train_feat_list),train_feat_list[0].shape,train_label.shape)
# 合并feat_list中的feature matrix，[sample_nb,feat_dim] -> [sample_nb,feat_dim*5]
train_feat = np.concatenate(train_feat_list,axis=1) 
train_feat = torch.from_numpy(train_feat).float()
train_label = torch.from_numpy(train_label).long() 
# label from [sample_nb,1] -> [sample_nb,class_nb]
#train_label = torch.zeros(train_label.shape[0],cls_nb).scatter_(1,train_label,1)
print(train_feat.shape,train_label.shape)

# get Dataset and DataLoader
from torch.utils.data import TensorDataset,DataLoader
trainset = TensorDataset(train_feat,train_label)

trainloader = DataLoader(trainset,batch_size=DIM,shuffle=False)


# get model
model = FuseNet(in_dim=DIM*5, dropout = 0., h_dim = DIM, out_dim = 1, )


# load model
model.load_state_dict(torch.load(ckpt_folder))

# get features
model.eval()
feat_layer = "fc1"
all_feat = []
all_label = []
all_pred = []
all_prob = []
with torch.no_grad():
    for data in trainloader:
        inputs,label = data
        feat = model(inputs,out_key=feat_layer)
        all_feat.append(feat)
        all_label.append(label)
        pred = model(inputs,out_key="all")
        prob = pred#F.softmax(pred,dim=1)
        all_prob.append(prob)

embedding = torch.cat(all_feat,dim=0).detach().numpy()
labels = torch.cat(all_label,dim=0).detach().numpy()
prob = torch.cat(all_prob,dim=0).detach().numpy()
print(embedding.shape,labels.shape,prob.shape)

5 (1980, 128) (1980, 1)
torch.Size([1980, 640]) torch.Size([1980, 1])
(1980, 1) (1980, 1) (1980, 1)


In [5]:
import sys
sys.path.append(code_loc)
import scLLM

# 数据集读取
import dill
# 用dill打开loc0的pkl 文件读取dataset
with open(raw_data_loc,"rb") as f:
    [dataset_1,dataset_2,_,_] = dill.load(f)
# 输出数据集信息
print("trainset size: ",len(dataset_1))

print(label_dict)
# reverse label_dict
rev_label_dict = {v:k for k,v in label_dict.items()}
print(rev_label_dict)
vocab = "/home/shi/WorkSpace/projects/scLLM_workspace/vocab_16k.json"
# read vocab
import json
with open(vocab,"r") as f:
    gene_names = json.load(f)
# 获取vocab按照value排序的key 的 list
#gene_names = [k for k,v in sorted(vocab.items(),key=lambda x:x[1])]
    
import scanpy as sc
# 第 1 步: 读取数据
exp_raw_data = dataset_1.data

root_label_value = label_dict["fake_lable_0"]
# 第 2 步：数据准备
adata = sc.AnnData(X=exp_raw_data)
adata.var_names = [name for name, _ in sorted(gene_names.items(), key=lambda x: x[1])]
adata.obs['labels'] = labels
adata.obsm['X_emb'] = embedding  # embedding 是嵌入矩阵

adata.obs["sample_id"] = dataset_1.sample_id


l_list = adata.obs['labels'].to_list()
l_list = [rev_label_dict[i] for i in l_list]
adata.obs['GT'] = l_list

# preds to str with label_dict
#pred_name = [rev_label_dict[i] for i in preds]
adata.obs["pred_labels"] = prob*10 #preds *10

trainset size:  1980
{'fake_lable_0': 0, 'fake_lable_1': 1}
{0: 'fake_lable_0', 1: 'fake_lable_1'}


In [6]:
# export to csv

data_dict = {
    "sample_id": adata.obs["sample_id"].to_list(),
    "pred_labels": adata.obs["pred_labels"].to_list(),
}
import pandas as pd
df = pd.DataFrame(data_dict)

df.to_csv(save_folder+"/metabric_predicted_labels_with_samples.csv",index=False)

In [7]:
# load model and calc features
import torch
import torch.nn.functional as F
import sys
DIM = 128


import torch.nn as nn

class FuseNet(nn.Module):
    def __init__(self,in_dim=128*5, dropout = 0., h_dim = 128, out_dim = 1, ):
        nn.Module.__init__(self,)

        self.fc1 = nn.Linear(in_features=in_dim, out_features=512, bias=True)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(in_features=512, out_features=h_dim, bias=True)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(in_features=h_dim, out_features=out_dim, bias=True)

    def forward(self, x, out_key:str = "all"):

        x = self.fc1(x)
        x = self.act1(x)
        x = self.dropout1(x)
        if out_key == "fc1":
            return x

        x = self.fc2(x)
        x = self.act2(x)
        x = self.dropout2(x)
        if out_key == "fc2":
            return x
        x = self.fc3(x)
        if out_key == "all":
            return x
        else:
            raise ValueError("out_key must be one of 'fc1', 'fc2', 'all'")

# 从文件夹中读取所有类的feature matrix
import pickle
import numpy as np


read_list = ["cls0","cls1","cls2","cls3","cls4"]
#-->train
train_feat_list = []
for i in range(len(read_list)):
    with open(train_root+read_list[i]+".pkl","rb") as f:
        [feat,label] = pickle.load(f)
        train_feat_list.append(feat)
train_label = label
print(len(train_feat_list),train_feat_list[0].shape,train_label.shape)
# 合并feat_list中的feature matrix，[sample_nb,feat_dim] -> [sample_nb,feat_dim*5]
train_feat = np.concatenate(train_feat_list,axis=1) 
train_feat = torch.from_numpy(train_feat).float()
train_label = torch.from_numpy(train_label).long() 
# label from [sample_nb,1] -> [sample_nb,class_nb]
#train_label = torch.zeros(train_label.shape[0],cls_nb).scatter_(1,train_label,1)
print(train_feat.shape,train_label.shape)

# get Dataset and DataLoader
from torch.utils.data import TensorDataset,DataLoader
trainset = TensorDataset(train_feat,train_label)

trainloader = DataLoader(trainset,batch_size=DIM,shuffle=False)


# get model
model = FuseNet(in_dim=DIM*5, dropout = 0., h_dim = DIM, out_dim = 5, )


# load model
model.load_state_dict(torch.load(ckpt_folder_cls))

# get features
model.eval()
feat_layer = "fc1"
all_feat = []
all_label = []
all_pred = []
all_prob = []
with torch.no_grad():
    for data in trainloader:
        inputs,label = data
        feat = model(inputs,out_key=feat_layer)
        all_feat.append(feat)
        all_label.append(label)
        pred = model(inputs,out_key="all")
        prob = pred#F.softmax(pred,dim=1)
        all_prob.append(prob)

embedding = torch.cat(all_feat,dim=0).detach().numpy()
labels = torch.cat(all_label,dim=0).detach().numpy()
prob = torch.cat(all_prob,dim=0).detach().numpy()
print(embedding.shape,labels.shape,prob.shape)

5 (1980, 128) (1980, 1)
torch.Size([1980, 640]) torch.Size([1980, 1])
(1980, 512) (1980, 1) (1980, 5)


In [8]:
pred = np.argmax(prob,axis=1)

In [9]:
# export to csv

data_dict = {
    "sample_id": adata.obs["sample_id"].to_list(),
    "pred_labels": pred.squeeze(),
}
import pandas as pd
df = pd.DataFrame(data_dict)

df.to_csv(save_folder+"/metabric_predicted_cat_with_samples.csv",index=False)